In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Sat May  7 05:53:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/huggingface/transformers

     |████████████████████████████████| 79 kB 5.1 MB/s 
     |████████████████████████████████| 325 kB 14.9 MB/s 
     |████████████████████████████████| 1.2 MB 71.1 MB/s 
     |████████████████████████████████| 4.3 MB 38.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 136 kB 72.4 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 1.1 MB 51.9 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
     |████████████████████████████████| 2.0 MB 51.3 MB/s 
     |████████████████████████████████| 131 kB 74.9 MB/s 
     |████████████████████████████████| 57 kB 7.5 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 2.4 MB 43.4 MB/s 
     |████████████████████████████████| 357 kB 74.5 MB/s 
     |██████

In [ ]:
cd Context-Selection/sentence-transformers

/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/Context-Selection/sentence-transformers


In [ ]:
!pip install -r requirements.txt

<h2> Building ground data

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
import numpy as np
context = '''
the farmer <SEP> Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing.

"What are you doing, Cotton?!"

"I only wanted to be more like you".

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pretty and special, like you. We would never want you to be any other way". And with that, Cotton's mommy picked her up and dropped her into a big bucket of water. When Cotton came out she was herself again. Her sisters licked her face until Cotton's fur was all all dry.

"Don't ever do that again, Cotton!" they all cried. "Next time you might mess up that pretty white fur of yours and we wouldn't want that!"

Then Cotton thought, "I change my mind. I like being special". <s> What color was Cotton? <s> white <s> Where did she live? <s> in a barn <s> Did she live alone? <s> no <s> Who did she live with? <s> with her mommy and 5 sisters <s> What color were her sisters? <s> orange and white <s> Was Cotton happy that she looked different than the rest of her family? <s> no <s> What did she do to try to make herself the same color as her sisters? <s> she painted herself
'''

import nltk
nltk.download('punkt')

answer = context.split('<SEP>')[0].strip()
context_and_history = context.split('<SEP>')[1].strip()
purely_context = context.split('<SEP>')[1].split('<s>')[0].strip()
history = context.split('<SEP>')[1].split('<s>')[1:]
print(history)

sentences = nltk.sent_tokenize(purely_context)
answers = [history[i].strip() for i in range(len(history)) if i%2 == 1]
QA = [history[i].strip() + ' ' + history[i+1].strip() for i in range(len(history)) if i%2 == 0]

import pandas as pd
data = {}

answers.append(answer)
print(answers)
print(QA)

scores = []
similarity_scores = []
for sentence in sentences:
  data[sentence] = []
  _scores = []
  for answer in QA:
    query_embedding = model.encode(sentence)
    passage_embedding = model.encode([answer])
    score = util.cos_sim(query_embedding, passage_embedding).detach().numpy()[0][0]
    _scores.append(score)
    data[sentence].append(score)
  scores.append(np.mean(np.array(data[sentence])))
  similarity_scores.append(_scores)
df = pd.DataFrame(data, index = QA)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[' What color was Cotton? ', ' white ', ' Where did she live? ', ' in a barn ', ' Did she live alone? ', ' no ', ' Who did she live with? ', ' with her mommy and 5 sisters ', ' What color were her sisters? ', ' orange and white ', ' Was Cotton happy that she looked different than the rest of her family? ', ' no ', ' What did she do to try to make herself the same color as her sisters? ', ' she painted herself\n']
['white', 'in a barn', 'no', 'with her mommy and 5 sisters', 'orange and white', 'no', 'she painted herself', 'the farmer']
['What color was Cotton? white', 'Where did she live? in a barn', 'Did she live alone? no', 'Who did she live with? with her mommy and 5 sisters', 'What color were her sisters? orange and white', 'Was Cotton happy that she looked different than the rest of her family? no', 'What did she do to try to make herself the same color as her sisters? she pain

,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton.",Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept.,"But Cotton wasn't alone in her little home above the barn, oh no.",She shared her hay bed with her mommy and 5 other sisters.,"All of her sisters were cute and fluffy, like Cotton.",But she was the only white one in the bunch.,The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy.,Being different made Cotton quite sad.,She often wished she looked like the rest of her family.,"So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them.",...,"Cotton's mommy rubbed her face on Cotton's and said ""Oh Cotton, but your fur is so pretty and special, like you.","We would never want you to be any other way"".","And with that, Cotton's mommy picked her up and dropped her into a big bucket of water.",When Cotton came out she was herself again.,Her sisters licked her face until Cotton's fur was all all dry.,"""Don't ever do that again, Cotton!""",they all cried.,"""Next time you might mess up that pretty white fur of yours and we wouldn't want that!""","Then Cotton thought, ""I change my mind.","I like being special""."
What color was Cotton? white,0.480543,0.433225,0.402306,0.305562,0.417169,0.331617,0.485056,0.602922,0.127067,0.474863,...,0.433608,0.043685,0.414142,0.464245,0.367873,0.500616,0.154917,0.207426,0.395581,0.015829
Where did she live? in a barn,0.529059,0.375838,0.554705,0.598347,0.310904,0.310453,0.304914,0.204349,0.341955,0.423282,...,0.308134,0.067764,0.409747,0.351765,0.263699,0.165932,0.120223,0.123191,0.119629,-0.018653
Did she live alone? no,0.245951,0.182723,0.483541,0.490659,0.230187,0.354303,0.252497,0.123185,0.337706,0.222700,...,0.167624,0.108709,0.225195,0.222890,0.216703,0.026648,0.147729,0.036677,0.059709,0.030759
Who did she live with? with her mommy and 5 sisters,0.375370,0.207477,0.403623,0.748190,0.418200,0.394953,0.456059,0.176492,0.412333,0.360264,...,0.324318,0.060965,0.422817,0.288114,0.370407,0.115516,0.270772,0.050730,0.062693,0.012524
What color were her sisters? orange and white,0.328312,0.170951,0.320361,0.578149,0.518081,0.570043,0.721296,0.287046,0.422778,0.541214,...,0.328934,0.050512,0.349503,0.304600,0.387493,0.179989,0.304601,0.194450,0.118207,0.051645
Was Cotton happy that she looked different than the rest of her family? no,0.381556,0.389730,0.607561,0.454371,0.506842,0.447748,0.541636,0.620172,0.548513,0.513806,...,0.554257,0.202439,0.489631,0.667739,0.462752,0.476153,0.225706,0.236662,0.491724,0.185531
What did she do to try to make herself the same color as her sisters? she painted herself,0.241186,0.087827,0.273304,0.474920,0.398772,0.440327,0.567522,0.338240,0.546441,0.706295,...,0.330750,0.067296,0.306671,0.438546,0.420232,0.188000,0.232347,0.267757,0.187936,0.117163


In [ ]:
"""
  return [[il, ir], [jl, jr]] such that similarity_scores[il:ir+1:][jl::jr+1::] >= a given p and
  S[il:ir+1:] contains Sl;
  H[jl:jr+1:] contains Ht;
  similarity_scores[i][j]: similarity between S_i and H_j;
  if there are two rectangle with similar perimeter -> return ones with higher score
  0 <= il, ir, jl, jr (the index value is w.r.t 0 index)
"""

def sentence_and_history_selection(sentences, histories, similarity_scores, p, idx_Sl, idx_Ht):
  ls, lh = len(sentences), len(histories)
  il, ir, jl, jr, np = 0, ls, 0, lh, 0
  dp = [[0 for i in range(lh)] for j in range(ls)]

  is_equal_sl = [1 if i == idx_Sl else 0 for i in range(ls)]
  ht_pos = [idx_Ht]

  if len(ht_pos) == 0: # no H position with value = Ht
    return sentences[il:ir+1], histories[jl:jr+1]

  if sum(is_equal_sl) == 0: # no S position with value = Sl
    return sentences[il:ir+1], histories[jl:jr+1]

  def get_sum(i1, j1, i2, j2): # get sum of dp[i1:i2+1:][j1:j2+1:] in O(1)
    r = dp[i2][j2]
    if i1 > 0:
      r -= dp[i1-1][j2]
    if j1 > 0:
      r -= dp[i2][j1-1]
    if i1 > 0 and j1 > 0:
      r += dp[i1-1][j1-1]
    return r

  for i in range(ls):
    for j in range(lh):
      if i > 0: dp[i][j] += dp[i-1][j]
      if j > 0: dp[i][j] += dp[i][j-1]
      if i > 0 and j > 0: dp[i][j] -= dp[i-1][j-1]
      dp[i][j] += similarity_scores[i][j]

  for i1 in range(ls): # O(ls)
    eq_sl_cnt = 0
    for i2 in range(i1, ls): # O(ls)
      eq_sl_cnt += is_equal_sl[i2]
      if eq_sl_cnt == 0: # don't care as this set of row don't contains the desired Sl value
        continue

      # apply two pointer technique again here (takes O(lh))
      j1, j2 = 0, 0
      p1, p2 = 0, 0
      if ht_pos[0] == 0:
        p2 += 1

      while j1 < lh:
        _np = get_sum(i1, j1, i2, j2)
        while j2 < lh and _np < p:
          j2 += 1
          if p2 < len(ht_pos) and j2 >= ht_pos[p2]:
            p2 += 1
          if j2 < lh:
            _np = get_sum(i1, j1, i2, j2)

        if _np >= p:
          cur_perimeter = (ir-il+1)+(jr-jl+1)

          if p2 - p1 > 0: # the segment contains the value = Ht
            upd_perimeter = (i2-i1+1)+(j2-j1+1)
            if cur_perimeter > upd_perimeter:
              il, ir, jl, jr, np = i1, i2, j1, j2, _np

            if cur_perimeter == upd_perimeter and _np >= np:
              il, ir, jl, jr, np = i1, i2, j1, j2, _np

          else:
            if p1 > 0:
              upd_perimeter = (i2-i1+1) + (j2-(p1-1)+1)
              _np = get_sum(il, ht_pos[p1-1], i2, j2)
              if cur_perimeter > upd_perimeter:
                il, ir, jl, jr, np = i1, i2, ht_pos[p1-1], j2, _np

              if cur_perimeter == upd_perimeter and _np >= np:
                il, ir, jl, jr, np = i1, i2, ht_pos[p1-1], j2, _np

            if p2 >= j2:
              if p2 == len(ht_pos):
                continue
              upd_perimeter = (i2 - i1 + 1) + (p2-j1+1)
              _np = get_sum(il, j1, i2, ht_pos[p2])
              if cur_perimeter > upd_perimeter:
                il, ir, jl, jr, np = i1, i2, j1, ht_pos[p2], _np

              if cur_perimeter == upd_perimeter and _np >= np:
                il, ir, jl, jr, np = i1, i2, j1, ht_pos[p2], _np

        else: # can't find any sub array with sum >= p anymore
          break

        j1 += 1
        if p1 < len(ht_pos) and j1 > ht_pos[p1]:
          p1 += 1

        if p1 == len(ht_pos):
          break

        if j1 > j2:
          j2 += 1
          if p2 < len(ht_pos) and ht_pos[p2] == j2:
            p2 += 1

  return sentences[il:ir+1], histories[jl:jr+1]
  # return sentences[il:ir+1], histories[jl:jr+1]

sentence_and_history_selection(sentences, QA, similarity_scores, 00, 5, 0)

(['But she was the only white one in the bunch.'],
 ['What color was Cotton? white'])

In [ ]:
import numpy as np

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def sentences_history_similarity(sentences, history):
  QA = [history[i].strip() + ' ' + history[i+1].strip() for i in range(len(history)) if i%2 == 0]
  scores = []
  data = {}
  for sentence in sentences:
    data[sentence] = []
    for answer in QA:
      query_embedding = model.encode(sentence)
      passage_embedding = model.encode([answer])
      score = util.dot_score(query_embedding, passage_embedding).detach().numpy()[0][0]
      data[sentence].append(score)
    scores.append(np.mean(np.array(data[sentence])))
  return scores, data

def sentence_selector(sentences, scores, idx, threshold):
  idx += 1
  n = len(scores)
  p = [0 for i in range(n+1)]
  poses = [idx]

  for i in range(1, n+1):
    p[i] = p[i-1] + scores[i-1]

  begin, end, sum = 1, n, 0
  for pos in poses: # two pointer to find the range with left < pos and right >= pos where sum(a[left:right+1:] >= s)
    l, r = 1, pos
    while l <= pos:
      while r <= n and p[r] - p[l-1] < threshold:
        r += 1
      if r == n+1:
        break
      else:
        ns = p[r] - p[l-1]
        if end - begin + 1 > r - l + 1:
          sum = ns
          begin, end = l, r
        elif end - begin + 1 == r - l + 1 and ns > sum:
          sum = ns
          begin, end = l, r
        l += 1
  return sentences[begin-1:end], begin-1, end-1

def longestCommonSubstring(s, t):
  n = len(s)
  m = len(t)
  dp = [[0 for i in range(m + 1)] for j in range(2)]
  res = 0
  for i in range(1,n + 1):
    for j in range(1,m + 1):
      if(s[i - 1] == t[j - 1]):
        dp[i % 2][j] = dp[(i - 1) % 2][j - 1] + 1
        if(dp[i % 2][j] > res):
          res = dp[i % 2][j]
      else:
        dp[i % 2][j] = 0
  return res

def find_index(span, sentences):
  lengths = []
  for sentence in sentences:
    lengths.append(longestCommonSubstring(span, sentence))
  return np.argmax(lengths)

def find_5_sentences(idx, sentences):
  num_sentences = len(sentences)
  if num_sentences <= 5:
    return sentences, 0, num_sentences - 1
  elif idx >= 2 and idx + 3 <= num_sentences:
    return [
            sentences[idx - 2],
            sentences[idx - 1],
            sentences[idx],
            sentences[idx + 1],
            sentences[idx + 2]
    ], idx - 2, idx + 2
  elif idx < 2:
    return [
            sentences[0],
            sentences[1],
            sentences[2],
            sentences[3],
            sentences[4]
    ], 0, 4
  elif idx + 3 > num_sentences:
    return [
            sentences[num_sentences - 5],
            sentences[num_sentences - 4],
            sentences[num_sentences - 3],
            sentences[num_sentences - 2],
            sentences[num_sentences - 1]
    ], num_sentences - 5, num_sentences - 1

def find_3_sentences(idx, sentences):
  num_sentences = len(sentences)
  if num_sentences <= 3:
    return sentences
  elif idx >= 1 and idx + 1 <= num_sentences - 1:
    return [
            sentences[idx - 1],
            sentences[idx],
            sentences[idx + 1],
    ]
  elif idx < 1:
    return [
            sentences[0],
            sentences[1],
            sentences[2],
    ]
  elif idx + 2 > num_sentences:
    return [
            sentences[num_sentences - 3],
            sentences[num_sentences - 2],
            sentences[num_sentences - 1]
    ]

<h2> Context-History Selection

In [ ]:
context = '''
the farmer <SEP> Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing.

"What are you doing, Cotton?!"

"I only wanted to be more like you".

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pretty and special, like you. We would never want you to be any other way". And with that, Cotton's mommy picked her up and dropped her into a big bucket of water. When Cotton came out she was herself again. Her sisters licked her face until Cotton's fur was all all dry.

"Don't ever do that again, Cotton!" they all cried. "Next time you might mess up that pretty white fur of yours and we wouldn't want that!"

Then Cotton thought, "I change my mind. I like being special". <s> What color was Cotton? <s> white <s> Where did she live? <s> in a barn <s> Did she live alone? <s> no <s> Who did she live with? <s> with her mommy and 5 sisters <s> What color were her sisters? <s> orange and white <s> Was Cotton happy that she looked different than the rest of her family? <s> no <s> What did she do to try to make herself the same color as her sisters? <s> she painted herself
'''

def history_selector(histories, scores, threshold):
  if len(scores) == 0 or sum(scores) < threshold:
    return histories
  if scores[len(scores)-1] >= threshold:
    return []
  curr_sum = 0
  return_list = []
  idx = len(histories)-1
  while curr_sum < threshold:
    return_list.append(histories[idx])
    curr_sum += scores[idx]
    idx -= 1
  return_list.reverse()
  return return_list

READ_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/train_QA_max_5_turns_with_input_QG.csv'
WRITE_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/Ablation-studies-p-HS/train_HS.csv'

import nltk
nltk.download('punkt')
from tqdm import tqdm
import csv

rows = []
idx_ = 0
with open(READ_FILE) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  for row in tqdm(csvreader):
    context = row[1]
    span_text = row[3]

    answer = context.split('<SEP>')[0].strip()
    context_and_history = context.split('<SEP>')[1].strip()
    purely_context = context.split('<SEP>')[1].split('<s>')[0].strip()
    history = context.split('<SEP>')[1].split('<s>')[1:]

    sentences = nltk.sent_tokenize(purely_context)
    QA_pairs = [history[i].strip() + ' ' + history[i+1].strip() for i in range(len(history)) if i%2 == 0]

    scores = []
    for QA in QA_pairs:
      _scores = []
      for sentence in sentences:
        query_embedding = model.encode(sentence)
        passage_embedding = model.encode([QA])
        score = util.cos_sim(query_embedding, passage_embedding).detach().numpy()[0][0]
        _scores.append(score)
      scores.append(np.mean(np.array(_scores)))

    new_row = row
    for p_value in [0,1,2,3,4,5,6]:
      selected_histories = history_selector(QA_pairs, scores, p_value)
      new_history_text = ' <s> '.join(selected_histories)
      new_context = 'Answer: ' + answer + ', ' + span_text + ' Context: ' + purely_context + ' <s> ' + new_history_text
      new_row.append(new_context)
    rows.append(new_row)
    idx_ += 1
    if idx_ == 10:
      break

for p_value in [0,1,2,3,4,5,6]:
  header.append(f'context_HS_{p_value}')

with open(WRITE_FILE, 'w') as file:
  csvwriter = csv.writer(file)
  csvwriter.writerow(header)
  csvwriter.writerows(rows)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


9it [00:09,  1.02s/it]


In [ ]:
longestCommonSubstring(
    'licked her face',
    'She shared her hay bed with her mommy and 5 other sisters'
)

7

In [ ]:
# TEST CASES
context = '''
the farmer <SEP> Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing.

"What are you doing, Cotton?!"

"I only wanted to be more like you".

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pretty and special, like you. We would never want you to be any other way". And with that, Cotton's mommy picked her up and dropped her into a big bucket of water. When Cotton came out she was herself again. Her sisters licked her face until Cotton's fur was all all dry.

"Don't ever do that again, Cotton!" they all cried. "Next time you might mess up that pretty white fur of yours and we wouldn't want that!"

Then Cotton thought, "I change my mind. I like being special". <s> What color was Cotton? <s> white <s> Where did she live? <s> in a barn <s> Did she live alone? <s> no <s> Who did she live with? <s> with her mommy and 5 sisters <s> What color were her sisters? <s> orange and white <s> Was Cotton happy that she looked different than the rest of her family? <s> no <s> What did she do to try to make herself the same color as her sisters? <s> she painted herself
'''

import nltk
nltk.download('punkt')

answer = context.split('<SEP>')[0].strip()
context_and_history = context.split('<SEP>')[1].strip()
purely_context = context.split('<SEP>')[1].split('<s>')[0].strip()
history = context.split('<SEP>')[1].split('<s>')[1:]
print(history)
sentences = nltk.sent_tokenize(purely_context)

import pandas as pd
scores, data = sentences_history_similarity(sentences, history)
# df = pd.DataFrame(data, index = QA)
print(sentences)
print(scores)
selected_sentences, _, _ = sentence_selector(sentences, scores, 3, 1.5)
print(selected_sentences)
sentence_score = []
for i in range(len(sentences)):
  sentence_score.append(sentences[i] + ': ' + str(scores[i]))
sentence_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[' What color was Cotton? ', ' white ', ' Where did she live? ', ' in a barn ', ' Did she live alone? ', ' no ', ' Who did she live with? ', ' with her mommy and 5 sisters ', ' What color were her sisters? ', ' orange and white ', ' Was Cotton happy that she looked different than the rest of her family? ', ' no ', ' What did she do to try to make herself the same color as her sisters? ', ' she painted herself\n']
['Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton.', "Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept.", "But Cotton wasn't alone in her little home above the barn, oh no.", 'She shared her hay bed with her mommy and 5 other sisters.', 'All of her sisters were cute and fluffy, like Cotton.', 'But she was the only white one in the bunch.', "The rest of her sisters were all ora

['Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton.: 0.36885378',
 "Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept.: 0.26396742",
 "But Cotton wasn't alone in her little home above the barn, oh no.: 0.43505716",
 'She shared her hay bed with her mommy and 5 other sisters.: 0.521457',
 'All of her sisters were cute and fluffy, like Cotton.: 0.4000221',
 'But she was the only white one in the bunch.: 0.4070632',
 "The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy.: 0.47556823",
 'Being different made Cotton quite sad.: 0.33605796',
 'She often wished she looked like the rest of her family.: 0.39097026',
 "So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them.: 0.463203",
 'When her mommy and sisters found her they started laughing.: 0.3812067',
 '"What are you doing, Cotton?!": 0.23952363',
 '"I o

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def calculate_similarity_scores(sentences, QA):
  similarity_scores = []
  for sentence in sentences:
    _scores = []
    for answer in QA:
      query_embedding = model.encode(sentence)
      passage_embedding = model.encode([answer])
      score = util.cos_sim(query_embedding, passage_embedding).detach().numpy()[0][0]
      _scores.append(score)
    similarity_scores.append(_scores)
  return similarity_scores

In [ ]:
import csv
from tqdm import tqdm

# Functions
import nltk
nltk.download('punkt')
import random

def process_context(span, context):
  answer = context.split('<SEP>')[0].strip()

  # if answer in ['YES', 'NO', 'Yes', 'No', 'yes', 'no', 'unknown', 'Unknown', 'UNKNOWN']:
  answer = answer +', ' + span

  context_and_history = context.split('<SEP>')[1].strip()

  main_context = context.split('<SEP>')[1].split(' <s> ')[0]
  QA_pairs = context.split('<SEP>')[1].split(' <s> ')[1:]

  purely_context = main_context.replace('\n', " ")
  sentences = nltk.sent_tokenize(purely_context)

  idx_S = -1
  idx_S = find_index(span, sentences)
  if idx_S == -1: raise Exception("Cannot find span!")

  QA = [QA_pairs[i] + ' '+ QA_pairs[i+1] for i in range(len(QA_pairs)) if i%2 ==0]
  top_p = 1
  contexts = []
  number_of_context_sentences = []
  number_of_history_turns = []

  if len(QA) != 0:
    similarity_scores = calculate_similarity_scores(sentences, QA)
  else:
    similarity_scores = []
  for top_p in [1,2,3,4,5,7,10]:
    if len(QA) == 0:
      selected_sentences = find_3_sentences(idx_S, sentences)
      selected_histories = []
    else:
      selected_sentences, selected_histories = sentence_and_history_selection(sentences, QA, similarity_scores, top_p, idx_S, len(QA) - 1)

    number_of_context_sentences.append(len(selected_sentences))
    number_of_history_turns.append(len(selected_histories))
    selected_context = ' '.join(selected_sentences)
    selected_history = ' <s> '.join(selected_histories)

    final_context = 'Answer: ' + answer + ' Context: ' + selected_context + ' <s> ' + selected_history
    contexts.append(final_context)

    assert len(contexts) == len(number_of_context_sentences)
    assert len(contexts) == len(number_of_history_turns)
  return contexts, number_of_context_sentences, number_of_history_turns

# Main file management
TRAIN_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/train_QA_max_5_turns_with_input_QG.csv'
TEST_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/valid_QA_max_5_turns_with_input_QG.csv'

WRITE_TRAIN_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/Ablation-studies-dynamic-context-history/train-dynamic-context-history.csv'
WRITE_VALID_FILE = '/content/gdrive/MyDrive/A/factoid_one_focus/CoQA/T5-BART/Ablation-studies-dynamic-context-history/valid-dynamic-context-history.csv'

train_rows = []
dev_rows = []

idx_ = 0
with open(TRAIN_FILE) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  for row in tqdm(csvreader):
    span_text = row[3]
    context = row[1]
    contexts, number_of_context_sentences, number_of_history_turns = process_context(span_text, context)
    new_row = row
    for idx in range(len(contexts)):
      new_row.extend([
                      contexts[idx],
                      number_of_context_sentences[idx],
                      number_of_history_turns[idx]
      ])
    train_rows.append(new_row)

    idx_ += 1
    if idx_ == 5: break

idx_ = 0
with open(TEST_FILE) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  for row in tqdm(csvreader):
    span_text = row[3]
    context = row[1]
    contexts, number_of_context_sentences, number_of_history_turns = process_context(span_text, context)
    new_row = row
    for idx in range(len(contexts)):
      new_row.extend([
                      contexts[idx],
                      number_of_context_sentences[idx],
                      number_of_history_turns[idx]
      ])
    dev_rows.append(new_row)

    idx_ += 1
    if idx_ == 5: break

for i in [1,2,3,4,5,7,10]:
  header.extend([
                 f'processed_context_p_{i}',
                 f'number_of_context_sentences_p_{i}',
                 f'number_of_history_turns_p_{i}',
  ])

with open(WRITE_TRAIN_FILE, 'w') as file:
  csvwriter = csv.writer(file)
  csvwriter.writerow(header)
  csvwriter.writerows(train_rows)

with open(WRITE_VALID_FILE, 'w') as file:
  csvwriter = csv.writer(file)
  csvwriter.writerow(header)
  csvwriter.writerows(dev_rows)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


4it [00:02,  1.75it/s]
4it [00:04,  1.06s/it]
